In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.datasets.utils import extract_archive
from torchvision.transforms import v2
from pathlib import Path

import sys

sys.path.append("../src")

from trainer import Trainer

In [2]:
data_path = Path("../data")
# extract_archive(data_path / "cifar-10-python.tar.gz")

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
trainer = Trainer(device=device)
trainer.load_model(
    run_path="sampath017/ImageClassification/ofrc6h0p",
    model_path="sampath017/ImageClassification/run-ofrc6h0p-model_19.pt:v0"
)

trainer.model

wandb:   1 of 1 files downloaded.  


ResNet(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0), dilation=1, ceil_mode=False)
    (7): ResidualBlock(
      (res_block): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
      )
    )
    (8): Conv2d(128, 256, kernel_size=

In [5]:
val_transforms = v2.Compose([
    # Normalize
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
])

test_dataset = CIFAR10(data_path, train=False, transform=val_transforms)

test_dataloader = DataLoader(test_dataset, batch_size=trainer.config["batch_size"])

In [6]:
loss, accuracy = trainer.test(test_dataloader)
print(f"{loss=:.2f}, {accuracy=:.2f}")

(tensor(0.3151), tensor(89.3778))

In [19]:
# api = wandb.Api()
# run = api.run("sampath017/ImageClassification/ofrc6h0p")
# config = run.config

# artifact = api.artifact("sampath017/ImageClassification/run-ofrc6h0p-model_19.pt:v0")
# local_path = artifact.download()
# model = ResNet()
# model.load_state_dict(torch.load(Path(local_path)/"model_19.pt", map_location=device))
# model

{'epochs': 20,
 'optimizer': {'name': 'Adam', 'max_lr': 0.01, 'weight_decay': 0.0001},
 'val_split': 8000,
 'batch_size': 1024,
 'train_split': 42000,
 'model_architecture': 'ResNet'}